In [2]:
import pygame
import cv2
import mediapipe as mp

pygame.init()


SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Gesture Controlled Game")


clock = pygame.time.Clock()
FPS = 30
player_pos = [400, 300]
player_speed = 5

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

def detect_hand_gesture(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * SCREEN_WIDTH)
            y = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * SCREEN_HEIGHT)
            return x, y
    return None


running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    ret, frame = cap.read()
    if not ret:
        break
    
    gesture_pos = detect_hand_gesture(frame)
    if gesture_pos:
        player_pos[0] = gesture_pos[0]
        player_pos[1] = gesture_pos[1]

    screen.fill((0, 0, 0))
    
    pygame.draw.circle(screen, (0, 255, 0), player_pos, 20)
    
    
    pygame.display.flip()
    

    clock.tick(FPS)

cap.release()
cv2.destroyAllWindows()
pygame.quit()
